After using Random Forest, K-Nearest Neighbors (KNN) and XGBoost algorithms, it has been concluded that KNN works best. So, in this notebook, we will use KNN as our final model and we will also test using the test data.

In [2]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

import cudf
import cuml
import cupy as cp
import numpy as np

In [4]:
import pandas as pd
X = pd.read_csv("../input/nepal-earthquake-complete/complete_x.csv")
y = pd.read_csv("../input/nepal-earthquake-complete/complete_y.csv")

In [5]:
X = X.astype("float32")
y = y.astype("float32")
y.set_index("building_id", inplace = True)
y = y - 1
y.head()

In [6]:

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify= y, test_size= 0.2,
                                                   random_state= 7)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, 
                                                      y_train, stratify= y_train, 
                                                      test_size= 0.2,
                                                     random_state= 7)

In [10]:
#KNN
knn = cuml.neighbors.KNeighborsClassifier(random_state = 7)
knn_params = { 'n_neighbors' : [25, 27, 29, 31, 33, 35],
               'weights' : ['uniform'],
               'metric' : ['manhattan']}

#Using grid search to find the best parameters
rf_random = GridSearchCV(estimator = knn, 
                               param_grid = knn_params, 
                               cv = 3, verbose=2, 
                               n_jobs = -1)
rf_random.fit(X_train, y_train)
rf_random.best_params_


In [12]:
#Retraining the model using best parameters
model = cuml.neighbors.KNeighborsClassifier(n_neighbors = 31,
                                                  weights= "uniform",
                                                  metric = "manhattan",
                                                  random_state = 7)
model.fit(X_train, y_train)
train_accuracy = model.score(X_train, y_train)
validation_accuracy = model.score(X_valid, y_valid)
print("The training accuracy is: ", train_accuracy)
print("The validation accuracy is: ", validation_accuracy)

In [15]:
test_accuracy = model.score(X_test, y_test)
print("The test accuracy is: ", round(test_accuracy, 3))


In [18]:
y_test.value_counts(normalize = True)

So, the test accuracy is 0.571 which means that our model predicts the correct category 57.1% of the time. 

It is worth mentioning that there are 5 different categories. And the probability of each category is different. The highest damage category which has been changed to 4 from 5, has a probability of 36.26%.

So, the model performance is average. 
